Universidade Federal de Alagoas

Instituto de Computação


Disciplina: Sistemas Distribuídos

Semestre letivo: 2018.2

Professor: André Lage Freitas


# Trabalho Prático AB2

**Importante**. Leia atentamente as especificações pois só serão aceitos trabalhos que as obedecerem.


## Parte 1: Implementação de um monitor de carga da trabalho descentralizado

**Implementar uma versão modificada do detector de falha não confiável** (_unreliable failure detector_) descrito na Seção _15.1.1 Failure assumptions and failure detectors_ do livro a seguir:

Coulouris, Dollimore, Kindberg and Blair, _Distributed Systems: Concepts and Design_. Addison-Wesley, 2012.

A **modificação** consiste em enviar informações sobre a carga do sistema (_system workload_) em vez de informações sobre falhas que indicariam os processos suspeitos e não-suspeitos. Para isso, em vez de cada processo enviar mensagem do tipo `I'm p and I'm alive`, os processos enviarão a medição interna da sua carga de trabalho. 

Para simular a medição da carga de trabalho dos processos, utilize números aleatórios entre `0` e `1` com precisão de uma casa decimal apenas, por exemplo, `0.4`.

Utilize como período de envio das mensagens `2` segundos.

### Detalhes de implementação

Cada processo do sistema deverá ser implementado como um Julia Worker. O Master será apenas o gestor do experimento, ou seja, será responsável para criar os processos (Julia Workers) e iniciar o sistema. Uma vez iniciado, o sistema deve funcionar por si sem depender do Master; uma vez que o algoritmo é descentralizado.

Para fins de visualização do monitoramento da carga de trabalho, os **Workers devem enviar ao Master** periodicamente (a cada `2` segundos) uma mensagem com o estado dos workers que eles monitoram (incluindo ele mesmo). Essa mensagem deve ser impresso na tela do master da seguinte forma.

```julia
[...]
Workload from Worker 4: [0.4,0.9,0.7]
Workload from Worker 2: [0.4,0.9,0.7]
Workload from Worker 3: [0.4,0.9,0.7]
[...]
```

A última linha acima significa que as cargas de trabalho dos Workers monitorados pelo Worker 3 são `0.4,0.9,0.7`, ou seja, os Workers têm as seguintes carga de trabalho:

* Worker 2 = 0.4
* Worker 3 = 0.9
* Worker 4 = 0.7

Lembre-se de que, em Julia, o ID do Master é 1 e os Workers têm ID maiores que 1.

## Parte 2: Implementação de um algoritmo de Eleição

**Implementar o algoritmo de Eleição** baseado em anel (_ring-based election algorithm_) descrito na Seção _15.3 Elections_ do livro a seguir:

Coulouris, Dollimore, Kindberg and Blair, _Distributed Systems: Concepts and Design_. Addison-Wesley, 2012.

Considere as seguintes condições.

1. Uma eleição nova é disparada **quando a carga de trabalho do líder for maior ou igual a `0.8`** e **o Worker que identificou essa alta carga de trabalho do líder tenha carga de trabalho menor que `0.8`**.
1. Em caso de empate, o Worker com ID mais alto ganha a eleição.
1. Caso não haja Worker com carga de trabalho menor que `0.8` para liderar, escolhe-se o Worker com menor carga de trabalho.

Quando um líder for eleito, ele deve enviar a a seguinte mensagem ao Master:

```julia
[...]
New leader elected! Worker 2
[...]
```


### Suporte de programação distribuída em Julia

Para utilizar o suporte de programação distribuída em Julia, primeiro devemos carergar a biblioteca `Distributed`:

```julia
using Pkg
Pkg.add("Distributed")
using Distributed
```

A seguir, segue uma lista (não extensiva) de funções que poderão ajudar na programação distribuída para a implementação do trabalho:

```julia
addprocs
remotecall
@fetchfrom
@everywhere
sleep
@spawn
```

Para buscar uma rápida ajuda sobre as funções, utilize `?` na frente da função (ver exemplo abaixo). 

```julia
?addprocs
```

Utilize o [material do mini-curso do Professor](https://github.com/proflage/2018-julia-hands-on) para estudo e a [documentação oficial da linguagem Julia (1.x)](https://docs.julialang.org/en/v1/) para demais dúvidas.

## Código-fonte

Documente seu código em comentários no próprio código-fonte.

**Todo o seu código tem que estar colado na próxima célula**, mesmo se ele tenha sido feito em IDEs, células separadas, etc. O Professor testará a apenas o que estiver colado na próxima célula.

In [ ]:
# Espaço para escrever o código fonte.
using Pkg
Pkg.add("Distributed")
using Distributed

addprocs(3)

#Declarations, workloads, leader and currentWorkerElection
#The currentWorkerElection is false because in the beginning
#no worker is participating in the election.
#the leader begins with nrpocs (4 in this case)
@everywhere begin
    workloads = fill(0.0, nprocs()) 
    leader = nprocs()
    currentWorkerElection = false
end

#Updates workloads value in worker with id "idx"
@everywhere function setValue(idx, cont)
    global workloads[idx] = cont
end

#Calculates next worker's id
#If CurrentIdx == nprocs (4), idx is 2 (next worker)
@everywhere function nextCalc(currentIdx)
    if(currentIdx == nprocs()) 
        return 2
    end
    return currentIdx+1
end

#If currentWorkerElection is true, updates leader to currentLeader
#and currentWorkerElection to false 
#(indicating that they are not participating in an election)
#All workers are set to false to indicate that the election is over
@everywhere function changeLeader(currentLeader)
    if(currentWorkerElection == true)
        global currentWorkerElection = false
        global leader = currentLeader
    end
end

#Updates the leader value in each worker
@everywhere function endElection(currentLeader)
    for i in workers()
        @spawnat i changeLeader(currentLeader)
    end
end

#Run the ring-based election algorithm
#currentIdx -> Former worker Id
#currentWorkload -> The lowest workload so far (from the leader)
#currentLeader -> Leader Id so far
@everywhere function election(currentIdx, currentWorkload, currentLeader)

    next = nextCalc(currentIdx)

    global currentWorkerElection = true
    
    if currentIdx != currentLeader
        if workloads[currentIdx] < currentWorkload
            @spawnat next election(next, workloads[currentIdx], currentIdx)
        elseif workloads[currentIdx] == currentWorkload && currentIdx > currentLeader
            @spawnat next election(next, currentWorkload, currentIdx)
        else
            @spawnat next election(next, currentWorkload, currentLeader)
        end
    elseif currentIdx == currentLeader
        @spawnat 1 println("New leader elected! Worker ", currentIdx)

        endElection(currentIdx)
    end
end

#Check if another election is already happening and
#check if the condition is met to run the election
@everywhere function checkWorkload(idx, currentWorkload)

    #If an election is already taking place, the function ends
    #This is becaus if an election has already taken place
    #All currentWorkerElection are set to false (endElection -> changeLeader) 
    for i in workers()
        if(@fetchfrom i currentWorkerElection == true) return 0
        end
    end

    #Condition to run the election
    if(workloads[leader] >= 0.8 && workloads[idx] < 0.8 && currentWorkerElection == false)
        global next = nextCalc(idx)
        global currentWorkerElection = true
        @spawnat next election(next, currentWorkload, idx)
    end
end

#Function to form the output string
@everywhere function printWorkloads(id, workloadsAux)

    ans = string("Workload from Worker ", id, ": ");
    ans *= string("[")

    for i in 2:nprocs()
            ans *= string(workloadsAux[i])

        if(i <= nworkers()) 
            ans *= string(", ")
        end
    end
    ans *= string("]")

    return ans
end

#Function "main", where are generated the new values
#for each worker, called the function of setting the values
#and print function
@everywhere function main()

    idCur = myid()
    while true
        cont = round(rand(), digits = 1)

        for i in workers()
            @spawnat i setValue(idCur, cont)
        end
        sleep(2)

        remotecall(println, 1, printWorkloads(idCur, workloads))
        checkWorkload(idCur, workloads[idCur]);
    end
end

println("Leader: ", nprocs())

for i in workers()
    @spawnat i main()
end




## Entrega

As respostas deverão ser entregues na parte indicada desse arquivo reservada ao código fonte, no formato [IJulia Notebook](https://github.com/JuliaLang/IJulia.jl), que utiliza tecnologia [Jupyter](https://www.jupyter.org).

O programa deve ser implementado na linguagem de programação [**Julia** versão 1.x](https://julialang.org/downloads/). Seu trabalho será testado na [JuliaBox](https://juliabox.com). 

Baixe seu arquivo `.ipynb` e anexe-o ao Google Classroom. Não se esqueça de **testar seu arquivo `.ipynb` na JuliaBox** antes de enviá-lo.


### Forma 

O arquivo IJulia Notebook deverá ser entregue ao Professor **exclusivamente através do Google Classroom**.



A responsabilidade sobre a integridade do arquivo contendo trabalho é exclusivamente dos discentes. Serão ignorados os trabalhos cujos arquivos não conseguirem ser abertos pelo Professor.

### Prazos

O prazo de entrega está descrito no **Google Classroom**.


## Pontuação extra

O(a) discente que realizar mais tarefas, além do que foi especificado neste trabalho, o professor atribuirá de 0,5 a 1,0 ponto extra a depender da relevância da contribuição no programa. O critério será decidido pelo professor. 

O(a) discente deve indicar qual é a tarefa executada. Por exemplo, adição de funcionalidades, armazenamento de dados em arquivo, documentação de código, comentários em inglês sem erros ortográficos, etc.


**Plágio** A nota zero será atribuída caso haja qualquer tipo de cópia parcial ou integral assim como as devidas medidas legais. Leia a [cartilha sobre plágio](http://www.noticias.uff.br/arquivos/cartilha-sobre-plagio-academico.pdf).

## Disclaimer

Esse material foi elaborado pelo [Prof. André Lage Freitas](https://sites.google.com/a/ic.ufal.br/andrelage/) e está licenciado sob a licença _GNU General Public License v3.0_.